# 🧠 Exploration Complète des LLMs : Paramètres, Tokens et Génération

## 🎯 Objectifs de ce notebook

Bienvenue dans ce notebook pédagogique complet dédié à la compréhension approfondie des **Large Language Models (LLMs)**.

### Ce que vous allez apprendre :

1. 🔤 **Comprendre la tokenization** : Comment les modèles découpent et encodent le texte
2. 📊 **Visualiser les embeddings** : La représentation vectorielle des mots dans l'espace sémantique
3. 👁️ **Explorer l'attention** : Le mécanisme clé des Transformers (Q/K/V, multi-head)
4. ⚙️ **Maîtriser les paramètres de génération** : temperature, top_k, top_p, etc.
5. 🔬 **Expérimenter concrètement** : Tester l'impact de chaque paramètre
6. 📈 **Analyser quantitativement** : Perplexité et métriques de qualité
7. ⚠️ **Comprendre les limites** : Hallucinations, biais, bonnes pratiques

---

## 🤖 Qu'est-ce qu'un LLM ?

Un **Large Language Model** est un modèle d'IA entraîné sur d'énormes quantités de texte pour :

- **Prédire le prochain token** (mot ou sous-mot) dans une séquence
- **Générer du texte** de manière cohérente et contextuelle
- **Comprendre le langage naturel** grâce aux représentations vectorielles

### Architecture : Le Transformer

Les LLMs modernes (GPT, BERT, etc.) reposent sur l'architecture **Transformer** qui utilise :

- **Self-Attention** : Permet au modèle de pondérer l'importance de chaque mot par rapport aux autres
- **Embeddings** : Transformation de tokens en vecteurs denses
- **Couches empilées** : Apprentissage de représentations de plus en plus abstraites

### Pipeline de génération :

```
Texte d'entrée → Tokenization → Embeddings → Transformer (N layers)
    ↓
Logits → Softmax → Sampling/Décodage → Texte généré
```

---

## 🎓 Public cible

Ce notebook s'adresse aux **professionnels techniques** :
- Data Scientists
- ML Engineers
- Développeurs IA
- Chercheurs en NLP

**Prérequis** : Connaissances de base en Python, ML et NLP.

---

💡 **Note** : Ce notebook utilise `distilgpt2` pour des raisons de légèreté et de rapidité d'exécution sur CPU. Les concepts s'appliquent à tous les LLMs (GPT, LLaMA, etc.).

---

# 📦 2. Installation et Imports

## Installation des dépendances

Nous utilisons les bibliothèques suivantes avec des **versions fixées** pour garantir la compatibilité :

- **transformers** : HuggingFace pour charger les modèles pré-entraînés
- **torch** : Backend PyTorch pour les calculs
- **plotly** : Visualisations interactives
- **scikit-learn** : PCA et autres outils ML
- **umap-learn** : Réduction de dimensionnalité avancée
- **numpy** & **pandas** : Manipulation de données

⚠️ **Important** : Les versions sont fixées pour éviter les problèmes de compatibilité.

In [ ]:
# Installation des packages avec versions fixées
# Décommenter les lignes suivantes pour installer dans un environnement vide

# !pip install transformers==4.36.2
# !pip install torch==2.1.2
# !pip install plotly==5.18.0
# !pip install scikit-learn==1.3.2
# !pip install umap-learn==0.5.5
# !pip install numpy==1.24.3
# !pip install pandas==2.0.3

## Imports et configuration

In [ ]:
# Imports standard
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from typing import List, Dict, Tuple, Optional
import random

# Transformers & PyTorch
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    set_seed as transformers_set_seed
)

# Visualisation
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Réduction de dimensionnalité
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
try:
    import umap
    UMAP_AVAILABLE = True
except ImportError:
    UMAP_AVAILABLE = False
    print("⚠️ UMAP non disponible. Installez avec: pip install umap-learn==0.5.5")

# Configuration
print("✅ Imports réussis!")
print(f"📱 PyTorch version: {torch.__version__}")
print(f"🔧 Device disponible: {'GPU (CUDA)' if torch.cuda.is_available() else 'CPU'}")

---

# 🤖 3. Choix et Chargement du Modèle

## Pourquoi `distilgpt2` ?

Nous utilisons **DistilGPT-2** pour ce notebook car :

### ✅ Avantages
- **Léger** : ~82M de paramètres (vs 1.5B pour GPT-2 large)
- **Rapide** : Fonctionne bien sur CPU
- **Open-source** : Libre d'utilisation
- **Pédagogique** : Même architecture que GPT-2, mais plus accessible

### 📊 Compromis taille / performance

| Modèle | Paramètres | Mémoire RAM | CPU/GPU |
|--------|------------|-------------|--------|
| distilgpt2 | 82M | ~350 MB | ✅ CPU OK |
| gpt2 | 124M | ~550 MB | ✅ CPU OK |
| gpt2-medium | 355M | ~1.5 GB | ⚠️ CPU lent |
| gpt2-large | 774M | ~3.2 GB | ❌ GPU recommandé |

### 🔄 Alternatives possibles

Vous pouvez facilement remplacer par d'autres modèles :
```python
model_name = "gpt2"           # Modèle standard
model_name = "gpt2-medium"    # Plus puissant
model_name = "EleutherAI/gpt-neo-125M"  # Alternative
```

In [ ]:
# Configuration du modèle
MODEL_NAME = "distilgpt2"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"🔧 Chargement du modèle: {MODEL_NAME}")
print(f"📱 Device: {DEVICE}")

# Chargement du tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Important : GPT-2 n'a pas de pad_token par défaut
# On utilise eos_token comme pad_token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("✅ pad_token configuré = eos_token")

# Chargement du modèle avec options pour l'analyse
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    output_attentions=True,      # Pour récupérer les matrices d'attention
    output_hidden_states=True,   # Pour récupérer les embeddings intermédiaires
)

# Déplacer le modèle sur le device approprié
model = model.to(DEVICE)
model.eval()  # Mode évaluation (pas d'entraînement)

print(f"\n✅ Modèle chargé avec succès!")
print(f"📊 Nombre de paramètres: {model.num_parameters():,}")
print(f"📏 Taille du vocabulaire: {tokenizer.vocab_size:,}")
print(f"🔢 Nombre de couches: {model.config.n_layer}")
print(f"👥 Nombre de têtes d'attention: {model.config.n_head}")
print(f"📐 Dimension des embeddings: {model.config.n_embd}")
print(f"📏 Context window: {model.config.n_positions} tokens")

---

# 🔤 4. Tokens et Tokenization

## Qu'est-ce qu'un token ?

Un **token** est l'unité de base traitée par un LLM. Ce n'est **pas forcément un mot** !

### 📚 Définition

- Un token peut être :
  - Un mot entier : `"hello"` → `[15339]`
  - Un sous-mot : `"tokenization"` → `["token", "ization"]`
  - Un caractère : espaces, ponctuation
  - Un caractère spécial : `\n`, `\t`

### 🎯 Pourquoi tokenizer ?

1. **Vocabulaire fini** : Impossible d'avoir tous les mots du monde
2. **Gestion de l'inconnu** : Découpe les mots rares en morceaux connus
3. **Efficacité** : Représentation numérique optimale

### 🔢 Tokenization BPE (Byte Pair Encoding)

GPT-2 utilise **BPE** :
- Apprend les paires de caractères les plus fréquentes
- Fusionne itérativement pour créer un vocabulaire
- Résultat : ~50,000 tokens dans le vocabulaire

In [ ]:
# Exemples de tokenization
exemples_textes = [
    "Bonjour, comment allez-vous ?",
    "L'intelligence artificielle révolutionne le monde.",
    "Tokenization est un concept fondamental.",
    "2024 sera une année exceptionnelle!",
    "Machine Learning & Deep Learning"
]

print("🔍 Exemples de tokenization:\n")

for texte in exemples_textes:
    # Tokenization
    tokens = tokenizer.tokenize(texte)
    token_ids = tokenizer.encode(texte, add_special_tokens=False)
    
    print(f"📝 Texte: {texte}")
    print(f"   🔤 Tokens: {tokens}")
    print(f"   🔢 IDs: {token_ids}")
    print(f"   📊 Nombre de tokens: {len(tokens)}")
    print()

## 💰 Impact des tokens : Coût et Context Window

### Coût
- Les APIs LLM facturent **au nombre de tokens** (input + output)
- Exemple OpenAI : ~$0.002 / 1K tokens (GPT-3.5)
- ⚠️ Un texte peut contenir plus de tokens que de mots !

### Context Window
- **Limite du modèle** : Nombre maximum de tokens en entrée
- distilgpt2 : 1024 tokens
- GPT-3.5-turbo : 4096 tokens  
- GPT-4 : 8192 tokens (ou 32K/128K pour versions étendues)
- Claude : jusqu'à 100K tokens

### Visualisation des tokens

In [ ]:
# Analyse détaillée d'un texte
texte_analyse = """L'apprentissage automatique et l'intelligence artificielle transforment 
radicalement notre société. Les modèles de langage comme GPT révolutionnent 
la génération de texte et la compréhension du langage naturel."""

tokens = tokenizer.tokenize(texte_analyse)
token_ids = tokenizer.encode(texte_analyse, add_special_tokens=False)

# Créer un DataFrame pour visualisation
df_tokens = pd.DataFrame({
    'Position': range(len(tokens)),
    'Token': tokens,
    'Token_ID': token_ids,
    'Longueur': [len(t) for t in tokens]
})

print(f"📊 Analyse détaillée:\n")
print(f"Texte original: {len(texte_analyse)} caractères")
print(f"Nombre de mots: {len(texte_analyse.split())}")
print(f"Nombre de tokens: {len(tokens)}")
print(f"Ratio tokens/mots: {len(tokens)/len(texte_analyse.split()):.2f}")

print(f"\n📋 Premiers tokens:\n")
print(df_tokens.head(15))

# Visualisation de la distribution des longueurs
fig = px.histogram(
    df_tokens, 
    x='Longueur',
    title="Distribution de la longueur des tokens",
    labels={'Longueur': 'Nombre de caractères', 'count': 'Fréquence'},
    color_discrete_sequence=['#636EFA']
)
fig.update_layout(showlegend=False)
fig.show()

---

# 📊 5. Embeddings : La Représentation Vectorielle

## Qu'est-ce qu'un embedding ?

Un **embedding** est une représentation **vectorielle dense** d'un token dans un espace de dimension élevée.

### 🎯 Transformation : Token → Vecteur

```
Token "chat" (ID: 34534) → Vecteur [0.23, -0.45, 0.87, ..., 0.12]
                                      ↑
                                768 dimensions (pour distilgpt2)
```

### 🔑 Propriétés clés

1. **Similarité sémantique** : Mots similaires = vecteurs proches
   - "chat" et "chien" auront des embeddings proches
   - "chat" et "ordinateur" seront éloignés

2. **Contexte** : Dans les Transformers, les embeddings sont **contextuels**
   - Le mot "banque" aura des représentations différentes selon le contexte

3. **Apprentissage** : Les embeddings sont **appris** pendant l'entraînement

### 🧮 Rôle dans les LLMs

- **Couche d'entrée** : Transforme les token IDs en vecteurs
- **Représentation continue** : Permet les calculs du Transformer
- **Espace sémantique** : Capture les relations linguistiques

In [ ]:
# Extraction des embeddings
def get_embeddings(texte: str) -> Tuple[np.ndarray, List[str]]:
    """
    Extrait les embeddings d'un texte.
    
    Returns:
        embeddings: Array de shape (n_tokens, embedding_dim)
        tokens: Liste des tokens correspondants
    """
    # Tokenization
    inputs = tokenizer(texte, return_tensors="pt").to(DEVICE)
    tokens = tokenizer.tokenize(texte)
    
    # Forward pass avec extraction des hidden states
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Les hidden_states contiennent les embeddings de toutes les couches
    # hidden_states[0] = embeddings de la couche d'entrée
    # hidden_states[-1] = embeddings de la dernière couche
    embeddings = outputs.hidden_states[-1].squeeze(0).cpu().numpy()
    
    return embeddings, tokens

# Exemple d'extraction
texte_exemple = "Les modèles de langage sont puissants et fascinants"
embeddings, tokens = get_embeddings(texte_exemple)

print(f"🔍 Analyse des embeddings:\n")
print(f"Texte: {texte_exemple}")
print(f"Nombre de tokens: {len(tokens)}")
print(f"Shape des embeddings: {embeddings.shape}")
print(f"  → {embeddings.shape[0]} tokens")
print(f"  → {embeddings.shape[1]} dimensions")
print(f"\nPremier embedding (premier token):")
print(f"  Min: {embeddings[0].min():.3f}")
print(f"  Max: {embeddings[0].max():.3f}")
print(f"  Mean: {embeddings[0].mean():.3f}")
print(f"  Std: {embeddings[0].std():.3f}")

## 📈 Visualisation des Embeddings

Les embeddings sont en haute dimension (768D pour distilgpt2). Pour les visualiser, on utilise des techniques de **réduction de dimensionnalité** :

### 1️⃣ PCA (Principal Component Analysis)
- Linéaire, rapide
- Préserve la variance globale
- Bon pour un aperçu rapide

### 2️⃣ t-SNE (t-Distributed Stochastic Neighbor Embedding)
- Non-linéaire
- Préserve les structures locales
- Excellent pour la visualisation de clusters

### 3️⃣ UMAP (Uniform Manifold Approximation and Projection)
- Non-linéaire, plus rapide que t-SNE
- Préserve structure globale ET locale
- État de l'art pour la visualisation

In [ ]:
# Préparer un texte plus long pour la visualisation
texte_long = """L'intelligence artificielle transforme notre monde. Les machines apprennent 
à comprendre le langage naturel. Les réseaux de neurones profonds analysent les données. 
Les algorithmes optimisent les processus. La technologie évolue rapidement. 
Les chercheurs développent des modèles innovants. Les applications pratiques se multiplient.
L'avenir de l'IA est prometteur et fascinant."""

# Extraire les embeddings
embeddings_viz, tokens_viz = get_embeddings(texte_long)

print(f"📊 Données pour visualisation:")
print(f"  Tokens: {len(tokens_viz)}")
print(f"  Dimensions originales: {embeddings_viz.shape[1]}")

# Réduction avec PCA
pca = PCA(n_components=2)
embeddings_pca = pca.fit_transform(embeddings_viz)
print(f"\n✅ PCA: Variance expliquée: {pca.explained_variance_ratio_.sum():.1%}")

# Réduction avec t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(tokens_viz)-1))
embeddings_tsne = tsne.fit_transform(embeddings_viz)
print(f"✅ t-SNE: Réduction effectuée")

# Réduction avec UMAP (si disponible)
if UMAP_AVAILABLE and len(tokens_viz) > 15:
    umap_reducer = umap.UMAP(n_components=2, random_state=42, n_neighbors=min(15, len(tokens_viz)-1))
    embeddings_umap = umap_reducer.fit_transform(embeddings_viz)
    print(f"✅ UMAP: Réduction effectuée")
else:
    embeddings_umap = None

In [ ]:
# Visualisation interactive avec Plotly
def plot_embeddings_comparison(embeddings_pca, embeddings_tsne, embeddings_umap, tokens):
    """
    Crée une visualisation comparative des trois méthodes de réduction.
    """
    # Déterminer le nombre de subplots
    n_plots = 3 if embeddings_umap is not None else 2
    
    if n_plots == 3:
        fig = make_subplots(
            rows=1, cols=3,
            subplot_titles=('PCA', 't-SNE', 'UMAP'),
            horizontal_spacing=0.1
        )
    else:
        fig = make_subplots(
            rows=1, cols=2,
            subplot_titles=('PCA', 't-SNE'),
            horizontal_spacing=0.15
        )
    
    # PCA
    fig.add_trace(
        go.Scatter(
            x=embeddings_pca[:, 0],
            y=embeddings_pca[:, 1],
            mode='markers+text',
            text=tokens,
            textposition='top center',
            marker=dict(size=10, color=range(len(tokens)), colorscale='Viridis'),
            name='PCA'
        ),
        row=1, col=1
    )
    
    # t-SNE
    fig.add_trace(
        go.Scatter(
            x=embeddings_tsne[:, 0],
            y=embeddings_tsne[:, 1],
            mode='markers+text',
            text=tokens,
            textposition='top center',
            marker=dict(size=10, color=range(len(tokens)), colorscale='Viridis'),
            name='t-SNE'
        ),
        row=1, col=2
    )
    
    # UMAP (si disponible)
    if n_plots == 3:
        fig.add_trace(
            go.Scatter(
                x=embeddings_umap[:, 0],
                y=embeddings_umap[:, 1],
                mode='markers+text',
                text=tokens,
                textposition='top center',
                marker=dict(size=10, color=range(len(tokens)), colorscale='Viridis'),
                name='UMAP'
            ),
            row=1, col=3
        )
    
    fig.update_layout(
        title_text="Visualisation des Embeddings - Comparaison des Méthodes",
        showlegend=False,
        height=500,
        width=1400 if n_plots == 3 else 1000
    )
    
    return fig

# Créer et afficher la visualisation
fig = plot_embeddings_comparison(embeddings_pca, embeddings_tsne, embeddings_umap, tokens_viz)
fig.show()

print("\n💡 Interprétation:")
print("- Les tokens sémantiquement similaires sont proches dans l'espace")
print("- Les couleurs représentent la position dans le texte")
print("- PCA: Vue linéaire globale")
print("- t-SNE: Emphase sur les clusters locaux")
if embeddings_umap is not None:
    print("- UMAP: Meilleur équilibre structure globale/locale")

---

# 👁️ 6. Attention et Self-Attention

## Le Mécanisme d'Attention : Cœur des Transformers

L'**attention** est le mécanisme qui permet au modèle de **pondérer l'importance** de chaque token par rapport aux autres lors du traitement.

### 🔑 Les 3 Composantes : Q, K, V

Pour chaque token, on calcule trois vecteurs :

1. **Q (Query)** : "Qu'est-ce que je cherche ?"
   - Représente l'information que le token souhaite obtenir

2. **K (Key)** : "Qu'est-ce que je peux offrir ?"
   - Représente le contenu du token

3. **V (Value)** : "Quelle information je porte ?"
   - Représente la valeur/information réelle du token

### 📐 Calcul de l'Attention

```
1. Similarité : Score = Q · Kᵀ / √d_k
2. Normalisation : Attention_weights = softmax(Score)
3. Agrégation : Output = Attention_weights · V
```

### 🎭 Multi-Head Attention

Au lieu d'une seule attention, on calcule **plusieurs attentions en parallèle** ("têtes") :

- Chaque tête se spécialise sur différents aspects (syntaxe, sémantique, etc.)
- distilgpt2 : **12 têtes** d'attention
- Les sorties sont concaténées et projetées

### 🔄 Self-Attention

Dans un LLM, c'est de la **self-attention** : chaque token attend aux autres tokens de la **même séquence**.

In [ ]:
# Extraction des matrices d'attention
def get_attention_weights(texte: str):
    """
    Extrait les poids d'attention pour un texte.
    
    Returns:
        attentions: Tuple de tenseurs (n_layers, batch, n_heads, seq_len, seq_len)
        tokens: Liste des tokens
    """
    inputs = tokenizer(texte, return_tensors="pt").to(DEVICE)
    tokens = tokenizer.tokenize(texte)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    # attentions est un tuple de tenseurs, un par couche
    attentions = outputs.attentions
    
    return attentions, tokens

# Exemple
texte_attention = "L'intelligence artificielle transforme le monde"
attentions, tokens_att = get_attention_weights(texte_attention)

print(f"🔍 Analyse des matrices d'attention:\n")
print(f"Texte: {texte_attention}")
print(f"Tokens: {tokens_att}")
print(f"Nombre de couches: {len(attentions)}")
print(f"Shape d'une matrice d'attention: {attentions[0].shape}")
print(f"  → Batch size: {attentions[0].shape[0]}")
print(f"  → Nombre de têtes: {attentions[0].shape[1]}")
print(f"  → Longueur séquence: {attentions[0].shape[2]}")

## 🔥 Visualisation des Heatmaps d'Attention

Les **heatmaps** montrent quels tokens le modèle "regarde" pour comprendre chaque token.

### Interprétation
- **Lignes** : Token qui "attend" (Query)
- **Colonnes** : Tokens auxquels on prête attention (Key)
- **Couleur** : Intensité de l'attention (0 à 1)
- **Patterns typiques** :
  - Diagonale : Auto-attention
  - Vertical : Attention vers tokens spécifiques (ex: ponctuation)
  - Diffus : Attention distribuée sur tout le contexte

In [ ]:
def plot_attention_heatmap(attention_weights, tokens, layer_idx=0, head_idx=0):
    """
    Visualise une heatmap d'attention pour une couche et tête spécifiques.
    """
    # Extraire la matrice pour la couche et tête spécifiées
    attention_matrix = attention_weights[layer_idx][0, head_idx].cpu().numpy()
    
    fig = go.Figure(data=go.Heatmap(
        z=attention_matrix,
        x=tokens,
        y=tokens,
        colorscale='Viridis',
        text=np.round(attention_matrix, 2),
        texttemplate='%{text}',
        textfont={"size": 8},
        colorbar=dict(title="Attention")
    ))
    
    fig.update_layout(
        title=f"Matrice d'Attention - Couche {layer_idx} - Tête {head_idx}",
        xaxis_title="Tokens (Keys)",
        yaxis_title="Tokens (Queries)",
        width=800,
        height=700
    )
    
    return fig

# Visualisation pour la première couche, première tête
fig = plot_attention_heatmap(attentions, tokens_att, layer_idx=0, head_idx=0)
fig.show()

print("\n💡 Que voir dans cette heatmap:")
print("- Chaque cellule (i,j) = Attention du token i vers le token j")
print("- Les valeurs sont normalisées (somme = 1 par ligne)")
print("- Zones claires = forte attention")
print("- Zones sombres = faible attention")

In [ ]:
# Comparaison de plusieurs têtes et couches
def plot_multi_head_attention(attention_weights, tokens, layer_idx=0, n_heads=4):
    """
    Visualise plusieurs têtes d'attention côte à côte.
    """
    n_heads = min(n_heads, attention_weights[layer_idx].shape[1])
    
    fig = make_subplots(
        rows=1, cols=n_heads,
        subplot_titles=[f'Tête {i}' for i in range(n_heads)],
        horizontal_spacing=0.05
    )
    
    for head_idx in range(n_heads):
        attention_matrix = attention_weights[layer_idx][0, head_idx].cpu().numpy()
        
        fig.add_trace(
            go.Heatmap(
                z=attention_matrix,
                x=tokens,
                y=tokens if head_idx == 0 else [''] * len(tokens),
                colorscale='Viridis',
                showscale=(head_idx == n_heads - 1),
                colorbar=dict(title="Attention") if head_idx == n_heads - 1 else None
            ),
            row=1, col=head_idx + 1
        )
    
    fig.update_layout(
        title_text=f"Multi-Head Attention - Couche {layer_idx}",
        height=500,
        width=300 * n_heads
    )
    
    return fig

# Afficher 4 têtes de la première couche
fig = plot_multi_head_attention(attentions, tokens_att, layer_idx=0, n_heads=4)
fig.show()

print("\n🎯 Observation:")
print("- Chaque tête capture des patterns différents")
print("- Certaines têtes se concentrent sur la structure syntaxique")
print("- D'autres capturent les relations sémantiques")
print("- La diversité des têtes enrichit la représentation")

---

# ⚙️ 7. Paramètres de Génération

## Introduction

Les **paramètres de génération** contrôlent comment le modèle produit du texte. Ils ont un impact majeur sur :
- La qualité du texte
- La créativité vs déterminisme
- La cohérence
- La diversité des sorties

Comprendre ces paramètres est **essentiel** pour utiliser efficacement les LLMs.

## 🔢 1. `max_new_tokens`

**Définition** : Nombre maximum de tokens à générer.

### Comportement
- Limite la longueur de la sortie
- Le modèle peut s'arrêter avant si il génère un token de fin (`<eos>`)

### Cas d'usage
- Contrôle de la longueur des réponses
- Gestion des coûts (APIs payantes)
- Respect des limites du context window

### Valeurs typiques
- Résumé court : 50-100
- Paragraphe : 150-300
- Article : 500-1000

```python
generate(prompt, max_new_tokens=50)
```

## 🌡️ 2. `temperature`

**Définition** : Contrôle l'aléatoire dans la sélection des tokens.

### Comportement

La température modifie les probabilités avant le sampling :

```
P'(token_i) = exp(logit_i / T) / Σ exp(logit_j / T)
```

- **T → 0** : Distribution très "peaky" → Déterministe
- **T = 1** : Distribution originale du modèle
- **T > 1** : Distribution plus "plate" → Plus aléatoire

### Impact pratique

| Temperature | Effet | Cas d'usage |
|-------------|-------|-------------|
| 0.0 - 0.3 | Très déterministe, répétitif | Tâches factuelles, traductions |
| 0.5 - 0.7 | Équilibré, créatif mais cohérent | Usage général, chatbots |
| 0.8 - 1.2 | Créatif, diversifié | Brainstorming, fiction |
| > 1.5 | Très aléatoire, parfois incohérent | Génération artistique |

### Exemple
```python
# Déterministe
generate(prompt, temperature=0.1)

# Créatif
generate(prompt, temperature=1.0)
```

## 🔝 3. `top_k`

**Définition** : Ne considère que les `k` tokens les plus probables à chaque étape.

### Comportement

1. Trier les tokens par probabilité décroissante
2. Garder seulement les `k` premiers
3. Renormaliser les probabilités
4. Sampler parmi ces `k` tokens

### Impact

- **top_k faible (10-20)** : Conservatif, prévisible
- **top_k moyen (40-50)** : Équilibré
- **top_k élevé (100+)** : Plus de diversité

### Exemple
```python
generate(prompt, do_sample=True, top_k=50)
```

⚠️ **Limitation** : top_k est fixe, peut inclure des tokens très improbables dans certains contextes.

## 🎯 4. `top_p` (Nucleus Sampling)

**Définition** : Sampling dans le "noyau" de probabilité cumulée `p`.

### Comportement

1. Trier les tokens par probabilité décroissante
2. Calculer la probabilité cumulée
3. Garder les tokens jusqu'à atteindre `p` (ex: 0.9)
4. Sampler parmi ces tokens

### Avantage sur top_k

- **Dynamique** : Adapte le nombre de tokens selon la distribution
- Si le modèle est très sûr (1 token à 95%) → peu de tokens
- Si le modèle hésite → plus de tokens

### Valeurs recommandées

| top_p | Effet |
|-------|-------|
| 0.1-0.5 | Très conservatif |
| 0.7-0.9 | **Recommandé** pour la plupart des cas |
| 0.95+ | Maximum de diversité |

### Exemple
```python
generate(prompt, do_sample=True, top_p=0.9)
```

💡 **Best practice** : Utiliser `top_p` plutôt que `top_k` dans la plupart des cas.

## 🎲 5. `do_sample`

**Définition** : Active/désactive le sampling probabiliste.

### Comportement

- **`do_sample=False`** (défaut) : **Greedy decoding**
  - Toujours choisir le token le plus probable
  - Complètement déterministe
  - Ignoré `temperature`, `top_k`, `top_p`

- **`do_sample=True`** : **Sampling**
  - Échantillonner selon les probabilités
  - Permet la diversité
  - Nécessaire pour utiliser temperature, top_k, top_p

### Exemple
```python
# Greedy (déterministe)
generate(prompt, do_sample=False)

# Sampling (probabiliste)
generate(prompt, do_sample=True, temperature=0.8)
```

## 🔄 6. `num_return_sequences`

**Définition** : Nombre de séquences différentes à générer.

### Comportement

- Génère plusieurs complétions pour le même prompt
- **Nécessite `do_sample=True`** pour avoir de la diversité

### Cas d'usage

- Générer plusieurs options
- Sélectionner la meilleure
- A/B testing

### Exemple
```python
outputs = generate(
    prompt, 
    do_sample=True,
    num_return_sequences=3,
    temperature=0.8
)
```

## 🚫 7. `repetition_penalty`

**Définition** : Pénalise les tokens déjà générés pour éviter les répétitions.

### Comportement

- **Valeur > 1.0** : Pénalise les répétitions
- **Valeur = 1.0** : Pas de pénalité (défaut)
- **Valeur < 1.0** : Encourage les répétitions (rare)

### Valeurs recommandées

- 1.0 - 1.2 : Léger contrôle
- 1.2 - 1.5 : Contrôle modéré (recommandé)
- 1.5+ : Fort contrôle (peut devenir artificiel)

### Exemple
```python
generate(prompt, repetition_penalty=1.2)
```

## 🎯 8. `seed` (via transformers.set_seed)

**Définition** : Graine aléatoire pour la reproductibilité.

### Comportement

- Fixe les générateurs aléatoires de PyTorch, numpy, etc.
- Permet de reproduire exactement les mêmes résultats

### Cas d'usage

- Debug
- Tests automatisés
- Comparaisons équitables

### Exemple
```python
transformers_set_seed(42)
output = generate(prompt, do_sample=True)
# Même résultat à chaque exécution
```

## 📏 9. Context Window

**Définition** : Limite maximale du nombre de tokens (input + output).

### Limites par modèle

| Modèle | Context Window |
|--------|----------------|
| distilgpt2 | 1024 tokens |
| GPT-2 | 1024 tokens |
| GPT-3 | 4096 tokens |
| GPT-3.5-turbo | 4096 tokens |
| GPT-4 | 8192 / 32K / 128K |
| Claude 2 | 100K tokens |

### Gestion

```python
max_length = model.config.n_positions
input_length = len(tokenizer.encode(prompt))
max_new_tokens = max_length - input_length - safety_margin
```

⚠️ **Attention** : Dépasser la limite → Erreur ou troncature

## 📊 Tableau Récapitulatif

| Paramètre | Valeur par défaut | Impact | Quand l'utiliser |
|-----------|-------------------|--------|------------------|
| `max_new_tokens` | 20 | Longueur de sortie | Toujours |
| `temperature` | 1.0 | Créativité | Avec `do_sample=True` |
| `top_k` | 50 | Filtrage des tokens | Alternative à top_p |
| `top_p` | 1.0 | Filtrage adaptatif | **Recommandé** avec sampling |
| `do_sample` | False | Active le sampling | Pour diversité |
| `num_return_sequences` | 1 | Nombre de sorties | Pour options multiples |
| `repetition_penalty` | 1.0 | Contrôle répétitions | Si texte répétitif |

### 🎯 Configurations Recommandées

**1. Tâches factuelles / Traduction**
```python
do_sample=False  # Greedy
# OU
do_sample=True
temperature=0.3
top_p=0.9
```

**2. Usage général / Chatbot**
```python
do_sample=True
temperature=0.7
top_p=0.9
repetition_penalty=1.2
```

**3. Créativité / Fiction**
```python
do_sample=True
temperature=1.0
top_p=0.95
```

---

# 🛠️ 8. Fonctions Utilitaires

Créons des fonctions réutilisables pour faciliter nos expérimentations.

In [ ]:
def set_seed(seed: int = 42):
    """
    Configure la graine aléatoire pour la reproductibilité.
    
    Args:
        seed: Valeur de la graine (défaut: 42)
    """
    transformers_set_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    print(f"✅ Seed fixé à {seed}")

In [ ]:
def generate_text(
    prompt: str,
    max_new_tokens: int = 50,
    temperature: float = 1.0,
    top_k: int = 50,
    top_p: float = 0.9,
    do_sample: bool = True,
    num_return_sequences: int = 1,
    repetition_penalty: float = 1.0,
    seed: Optional[int] = None
) -> List[str]:
    """
    Génère du texte avec le modèle chargé.
    
    Args:
        prompt: Texte d'entrée
        max_new_tokens: Nombre maximum de tokens à générer
        temperature: Contrôle la créativité (0.0-2.0)
        top_k: Filtre top-k tokens
        top_p: Nucleus sampling (0.0-1.0)
        do_sample: Active le sampling probabiliste
        num_return_sequences: Nombre de séquences à générer
        repetition_penalty: Pénalité pour les répétitions
        seed: Graine aléatoire (optionnel)
    
    Returns:
        Liste des textes générés
    """
    if seed is not None:
        set_seed(seed)
    
    # Encoder le prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    
    # Générer
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature if do_sample else 1.0,
            top_k=top_k if do_sample else None,
            top_p=top_p if do_sample else None,
            do_sample=do_sample,
            num_return_sequences=num_return_sequences,
            repetition_penalty=repetition_penalty,
            pad_token_id=tokenizer.pad_token_id,
        )
    
    # Décoder
    generated_texts = [
        tokenizer.decode(output, skip_special_tokens=True)
        for output in outputs
    ]
    
    return generated_texts

# Test de la fonction
set_seed(42)
test_prompt = "L'intelligence artificielle est"
result = generate_text(test_prompt, max_new_tokens=20, temperature=0.7)
print(f"\n📝 Prompt: {test_prompt}")
print(f"🤖 Génération: {result[0]}")

In [ ]:
def get_next_token_probs(
    prompt: str,
    top_n: int = 10
) -> pd.DataFrame:
    """
    Calcule les probabilités des prochains tokens.
    
    Args:
        prompt: Texte d'entrée
        top_n: Nombre de top tokens à retourner
    
    Returns:
        DataFrame avec tokens et probabilités
    """
    # Encoder
    inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    
    # Forward pass
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Logits du dernier token
    logits = outputs.logits[0, -1, :]
    
    # Softmax pour obtenir les probabilités
    probs = torch.softmax(logits, dim=-1)
    
    # Top-N
    top_probs, top_indices = torch.topk(probs, top_n)
    
    # Créer DataFrame
    df = pd.DataFrame({
        'Token': [tokenizer.decode([idx]) for idx in top_indices.cpu().numpy()],
        'Token_ID': top_indices.cpu().numpy(),
        'Probabilité': top_probs.cpu().numpy(),
        'Probabilité_%': (top_probs.cpu().numpy() * 100)
    })
    
    return df

# Test
test_prompt = "Paris est la capitale de la"
probs_df = get_next_token_probs(test_prompt, top_n=10)
print(f"\n📊 Top 10 tokens pour: '{test_prompt}'\n")
print(probs_df.to_string(index=False))

---

# 🔬 9. Expérimentations Guidées

Testons concrètement l'impact des paramètres de génération.

## Expérience 1 : Impact de la Temperature

In [ ]:
# Prompt commun
prompt_exp = "L'avenir de l'intelligence artificielle sera"

# Tester différentes températures
temperatures = [0.0, 0.5, 0.7, 1.0, 1.5]
results_temp = {}

print(f"🔬 Expérience: Impact de la température\n")
print(f"📝 Prompt: '{prompt_exp}'\n")
print("=" * 80)

for temp in temperatures:
    # Générer 3 séquences pour voir la diversité
    outputs = generate_text(
        prompt_exp,
        max_new_tokens=30,
        temperature=temp if temp > 0 else 1.0,  # Éviter temp=0 exact
        do_sample=(temp > 0.01),
        num_return_sequences=3 if temp > 0.01 else 1,
        seed=42
    )
    
    results_temp[temp] = outputs
    
    print(f"\n🌡️  Temperature = {temp}")
    print("-" * 80)
    for i, output in enumerate(outputs, 1):
        print(f"   {i}. {output}")
    
    # Mesure de diversité : nombre de sorties uniques
    unique_outputs = len(set(outputs))
    print(f"   📊 Sorties uniques: {unique_outputs}/{len(outputs)}")

print("\n" + "=" * 80)
print("\n💡 Observations:")
print("- Temperature ≈ 0 : Très déterministe, toujours la même sortie")
print("- Temperature = 0.7 : Bon équilibre entre cohérence et diversité")
print("- Temperature élevée : Plus de variété, parfois moins cohérent")

## Expérience 2 : Visualisation des Probabilités

In [ ]:
# Analyser les probabilités avec différentes températures
prompt_prob = "Le capital de France est"

# Obtenir les probabilités de base
df_probs = get_next_token_probs(prompt_prob, top_n=15)

# Créer un bar chart
fig = px.bar(
    df_probs,
    x='Token',
    y='Probabilité_%',
    title=f"Distribution des probabilités pour: '{prompt_prob}'",
    labels={'Probabilité_%': 'Probabilité (%)'},
    color='Probabilité_%',
    color_continuous_scale='Viridis'
)

fig.update_layout(
    xaxis_tickangle=-45,
    height=500,
    showlegend=False
)

fig.show()

print("\n📊 Analyse:")
print(f"- Token le plus probable: '{df_probs.iloc[0]['Token']}' ({df_probs.iloc[0]['Probabilité_%']:.2f}%)")
print(f"- Entropie: {'Faible' if df_probs.iloc[0]['Probabilité_%'] > 50 else 'Élevée'}")
print(f"- Le modèle est {'très confiant' if df_probs.iloc[0]['Probabilité_%'] > 70 else 'incertain'}")

## Expérience 3 : Comparaison top_k vs top_p

In [ ]:
prompt_exp3 = "Dans le futur, les robots pourront"

print(f"🔬 Expérience: top_k vs top_p\n")
print(f"📝 Prompt: '{prompt_exp3}'\n")
print("=" * 80)

# Test avec top_k
print("\n🔝 Avec top_k=10")
outputs_topk = generate_text(
    prompt_exp3,
    max_new_tokens=25,
    do_sample=True,
    top_k=10,
    top_p=1.0,  # Désactiver top_p
    temperature=0.8,
    num_return_sequences=3,
    seed=42
)
for i, output in enumerate(outputs_topk, 1):
    print(f"   {i}. {output}")

# Test avec top_p
print("\n🎯 Avec top_p=0.9")
outputs_topp = generate_text(
    prompt_exp3,
    max_new_tokens=25,
    do_sample=True,
    top_k=0,  # Désactiver top_k (mettre à 0 ou très grand)
    top_p=0.9,
    temperature=0.8,
    num_return_sequences=3,
    seed=42
)
for i, output in enumerate(outputs_topp, 1):
    print(f"   {i}. {output}")

print("\n" + "=" * 80)
print("\n💡 Différence:")
print("- top_k : Fixe, peut être trop restrictif ou trop permissif")
print("- top_p : Adaptif, s'ajuste selon la confiance du modèle")
print("- top_p est généralement préféré pour un meilleur équilibre")

## Expérience 4 : Effet de repetition_penalty

In [ ]:
prompt_exp4 = "Les avantages de l'IA sont nombreux. Premièrement"

print(f"🔬 Expérience: Repetition Penalty\n")
print(f"📝 Prompt: '{prompt_exp4}'\n")
print("=" * 80)

# Sans pénalité
print("\n📝 Sans pénalité (1.0)")
output_no_penalty = generate_text(
    prompt_exp4,
    max_new_tokens=40,
    do_sample=True,
    temperature=0.8,
    repetition_penalty=1.0,
    seed=42
)[0]
print(f"   {output_no_penalty}")

# Avec pénalité modérée
print("\n🚫 Avec pénalité modérée (1.2)")
output_penalty = generate_text(
    prompt_exp4,
    max_new_tokens=40,
    do_sample=True,
    temperature=0.8,
    repetition_penalty=1.2,
    seed=42
)[0]
print(f"   {output_penalty}")

# Avec forte pénalité
print("\n⛔ Avec forte pénalité (1.5)")
output_high_penalty = generate_text(
    prompt_exp4,
    max_new_tokens=40,
    do_sample=True,
    temperature=0.8,
    repetition_penalty=1.5,
    seed=42
)[0]
print(f"   {output_high_penalty}")

print("\n" + "=" * 80)
print("\n💡 Impact:")
print("- Pénalité faible : Peut répéter des mots/phrases")
print("- Pénalité modérée : Bon équilibre, texte naturel")
print("- Pénalité forte : Évite les répétitions mais peut être artificiel")

---

# 🔄 10. Pipeline Complet d'une Requête

Illustrons le **parcours complet** d'un texte dans un LLM, étape par étape.

In [ ]:
# Texte d'exemple
user_text = "L'intelligence artificielle transforme"

print("="*80)
print("🔄 PIPELINE COMPLET : TEXTE → GÉNÉRATION")
print("="*80)

# ÉTAPE 1: Texte d'entrée
print(f"\n1️⃣  TEXTE UTILISATEUR")
print(f"   📝 Input: '{user_text}'")
print(f"   📏 Longueur: {len(user_text)} caractères")

# ÉTAPE 2: Tokenization
print(f"\n2️⃣  TOKENIZATION")
tokens = tokenizer.tokenize(user_text)
token_ids = tokenizer.encode(user_text, return_tensors="pt").to(DEVICE)
print(f"   🔤 Tokens: {tokens}")
print(f"   🔢 IDs: {token_ids.cpu().numpy()[0]}")
print(f"   📊 Nombre de tokens: {len(tokens)}")

# ÉTAPE 3: Embeddings
print(f"\n3️⃣  EMBEDDINGS")
with torch.no_grad():
    outputs_full = model(token_ids, output_hidden_states=True)
embeddings_input = outputs_full.hidden_states[0].squeeze().cpu().numpy()
print(f"   📐 Shape: {embeddings_input.shape}")
print(f"   → {embeddings_input.shape[0]} tokens")
print(f"   → {embeddings_input.shape[1]} dimensions par token")

# ÉTAPE 4: Transformer (N couches)
print(f"\n4️⃣  TRANSFORMER ({model.config.n_layer} couches)")
print(f"   🔄 Passage à travers {model.config.n_layer} couches")
print(f"   👁️  Chaque couche avec {model.config.n_head} têtes d'attention")
print(f"   📊 Hidden states à chaque couche")

# ÉTAPE 5: Logits
print(f"\n5️⃣  LOGITS (scores bruts)")
logits = outputs_full.logits[0, -1, :]  # Logits du dernier token
print(f"   📏 Shape: {logits.shape}")
print(f"   → Un score pour chaque token du vocabulaire ({tokenizer.vocab_size:,})")
print(f"   📈 Min: {logits.min():.2f}, Max: {logits.max():.2f}, Mean: {logits.mean():.2f}")

# ÉTAPE 6: Softmax → Probabilités
print(f"\n6️⃣  SOFTMAX → PROBABILITÉS")
probs = torch.softmax(logits, dim=-1)
print(f"   📊 Somme des probabilités: {probs.sum():.6f} (≈ 1.0)")
top5_probs, top5_indices = torch.topk(probs, 5)
print(f"   🏆 Top 5 tokens:")
for i, (prob, idx) in enumerate(zip(top5_probs, top5_indices), 1):
    token_str = tokenizer.decode([idx.item()])
    print(f"      {i}. '{token_str}' : {prob.item()*100:.2f}%")

# ÉTAPE 7: Sampling/Décodage
print(f"\n7️⃣  SAMPLING / DÉCODAGE")
print(f"   🎲 Sélection du prochain token selon les paramètres")
print(f"   🔄 Répétition du processus pour chaque nouveau token")

# ÉTAPE 8: Génération complète
print(f"\n8️⃣  TEXTE GÉNÉRÉ")
generated = generate_text(
    user_text,
    max_new_tokens=15,
    temperature=0.7,
    do_sample=True,
    seed=42
)[0]
print(f"   ✨ Résultat: '{generated}'")

print("\n" + "="*80)
print("\n💡 Points clés:")
print("- Chaque token généré devient input pour le suivant")
print("- Le processus est auto-régressif (un token à la fois)")
print("- Les paramètres affectent l'étape de sampling (étape 7)")
print("- Le modèle ne 'voit' que les tokens précédents (causal)")

---

# 📈 11. Analyse Quantitative : Perplexité

## Qu'est-ce que la Perplexité ?

La **perplexité** est une métrique qui mesure la **qualité d'un modèle de langage**.

### Définition mathématique

```
Perplexité = exp(- (1/N) * Σ log P(token_i | contexte))
```

Où :
- N = nombre de tokens
- P(token_i | contexte) = probabilité du token i donné le contexte

### Interprétation

- **Perplexité faible** : Le modèle est confiant, prédit bien
- **Perplexité élevée** : Le modèle est incertain

💡 **Intuition** : La perplexité représente le nombre moyen de choix équiprobables à chaque étape.

### Valeurs typiques

- GPT-2 (1.5B) : ~20-30 sur des textes standards
- Texte aléatoire : ~50,000 (taille du vocabulaire)
- Texte très prévisible : ~5-10

In [ ]:
def calculate_perplexity(text: str) -> float:
    """
    Calcule la perplexité approximative du modèle sur un texte.
    
    Args:
        text: Texte à évaluer
    
    Returns:
        Perplexité (float)
    """
    # Encoder le texte
    inputs = tokenizer(text, return_tensors="pt").to(DEVICE)
    input_ids = inputs['input_ids']
    
    # Forward pass
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
    
    # La loss est déjà la cross-entropy moyenne
    loss = outputs.loss
    
    # Perplexité = exp(loss)
    perplexity = torch.exp(loss).item()
    
    return perplexity

# Tests sur différents textes
textes_test = [
    ("Paris est la capitale de la France.", "Texte factuel simple"),
    ("L'intelligence artificielle transforme radicalement notre société moderne.", "Texte cohérent"),
    ("xkzp qwerty asdfgh zxcvbn uiop jklm", "Texte aléatoire"),
    ("Le le le le le le le le le le", "Répétitions"),
]

print("📊 Calcul de Perplexité sur différents textes\n")
print("="*80)

results_perplexity = []

for text, description in textes_test:
    perplexity = calculate_perplexity(text)
    results_perplexity.append({
        'Description': description,
        'Texte': text[:50] + '...' if len(text) > 50 else text,
        'Perplexité': perplexity
    })
    print(f"\n📝 {description}")
    print(f"   Texte: '{text}'")
    print(f"   📊 Perplexité: {perplexity:.2f}")

print("\n" + "="*80)

# Créer un DataFrame et visualiser
df_perplexity = pd.DataFrame(results_perplexity)

fig = px.bar(
    df_perplexity,
    x='Description',
    y='Perplexité',
    title="Comparaison des Perplexités",
    labels={'Perplexité': 'Perplexité', 'Description': 'Type de texte'},
    color='Perplexité',
    color_continuous_scale='RdYlGn_r'
)

fig.update_layout(showlegend=False, height=500)
fig.show()

print("\n💡 Interprétation:")
print("- Texte naturel et cohérent → Perplexité faible")
print("- Texte aléatoire → Perplexité très élevée")
print("- Répétitions → Perplexité faible (trop prévisible)")
print("\n🎯 La perplexité mesure la 'surprise' du modèle face au texte")

---

# ⚠️ 12. Bonnes Pratiques et Limites

## 🚨 Limites des LLMs

### 1. Hallucinations

Les LLMs peuvent **inventer des informations** qui semblent plausibles mais sont fausses.

**Causes** :
- Le modèle génère du texte plausible, pas nécessairement vrai
- Données d'entraînement incomplètes ou erronées
- Manque de connaissance du monde réel

**Mitigation** :
- ✅ Vérifier les faits importants
- ✅ Utiliser RAG (Retrieval Augmented Generation)
- ✅ Demander des sources
- ✅ Temperature plus basse pour tâches factuelles

### 2. Biais

Les LLMs reflètent les **biais présents dans leurs données d'entraînement**.

**Types de biais** :
- Biais sociaux (genre, race, culture)
- Biais temporels (données anciennes)
- Biais de représentation (sur/sous-représentation)

**Mitigation** :
- ✅ Être conscient des biais
- ✅ Tester sur des cas divers
- ✅ Utiliser des garde-fous
- ✅ Modération humaine

### 3. Context Window Limité

**Problème** : Le modèle ne peut traiter qu'un nombre limité de tokens.

- distilgpt2 : 1024 tokens (~750 mots)
- GPT-4 : 8K-128K tokens

**Solutions** :
- ✅ Découpage intelligent du texte
- ✅ Résumés progressifs
- ✅ Embeddings + recherche sémantique

### 4. Pas de Connaissance du Monde Réel

- ❌ Pas d'accès à Internet
- ❌ Pas de mise à jour en temps réel
- ❌ Pas de vérification des faits

**Solutions** :
- ✅ RAG avec base de connaissances
- ✅ Intégration d'APIs externes
- ✅ Fine-tuning sur données récentes

### 5. Coût Computationnel

**Ressources nécessaires** :
- Mémoire GPU importante
- Latence (temps de réponse)
- Coût monétaire (APIs)

**Optimisations** :
- ✅ Utiliser des modèles plus petits quand possible
- ✅ Quantization (réduction de précision)
- ✅ Batching des requêtes
- ✅ Caching des résultats

---

## ✅ Bonnes Pratiques

### 1. Prompt Engineering

```python
# ❌ Mauvais prompt
"Explique l'IA"

# ✅ Bon prompt
"""Explique l'intelligence artificielle en 3 paragraphes,
en utilisant des analogies simples pour un public non-technique.
Structure: 1) Définition, 2) Applications, 3) Défis."""
```

### 2. Choix des Paramètres

**Tâches factuelles** :
```python
temperature=0.3
top_p=0.9
do_sample=True
```

**Tâches créatives** :
```python
temperature=0.9
top_p=0.95
do_sample=True
```

### 3. Validation et Tests

- ✅ Tester sur des cas limites
- ✅ Mesurer la qualité (perplexité, cohérence)
- ✅ Évaluation humaine
- ✅ Monitoring en production

### 4. Sécurité des Prompts

**Risques** :
- Prompt injection
- Extraction de données sensibles
- Génération de contenu nuisible

**Protection** :
```python
# Validation des inputs
def sanitize_prompt(prompt: str) -> str:
    # Limiter la longueur
    max_length = 1000
    prompt = prompt[:max_length]
    
    # Filtrer les instructions dangereuses
    dangerous_patterns = ["ignore previous", "disregard"]
    # ... filtrage
    
    return prompt
```

### 5. Recommandations pour la Production

#### Infrastructure
- ✅ Load balancing
- ✅ Rate limiting
- ✅ Caching intelligent
- ✅ Monitoring et alertes

#### Qualité
- ✅ A/B testing des prompts
- ✅ Logging des générations
- ✅ Feedback utilisateurs
- ✅ Revue manuelle échantillonnée

#### Coûts
- ✅ Optimiser max_tokens
- ✅ Utiliser des modèles adaptés à la tâche
- ✅ Caching agressif
- ✅ Batch processing quand possible

---

## 🎓 Pour Aller Plus Loin

### Concepts Avancés

1. **Fine-tuning** : Adapter un modèle pré-entraîné à votre domaine
2. **RAG** : Retrieval Augmented Generation pour données factuelles
3. **Agents** : LLMs avec outils et planification
4. **Multi-modal** : Modèles texte + images

### Ressources

- 📚 [HuggingFace Documentation](https://huggingface.co/docs)
- 📄 [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
- 🎓 [Stanford CS224N: NLP with Deep Learning](http://web.stanford.edu/class/cs224n/)
- 🔧 [OpenAI Cookbook](https://github.com/openai/openai-cookbook)

### Outils

- **LangChain** : Framework pour applications LLM
- **LlamaIndex** : Framework pour RAG
- **PEFT** : Parameter-Efficient Fine-Tuning
- **vLLM** : Serving rapide de LLMs

---

## 🎉 Conclusion

### Ce que vous avez appris

✅ **Tokens & Tokenization** : Comment les LLMs découpent le texte  
✅ **Embeddings** : Représentation vectorielle et visualisation  
✅ **Attention** : Le mécanisme clé des Transformers  
✅ **Paramètres de génération** : Temperature, top_k, top_p, etc.  
✅ **Expérimentation** : Impact pratique des paramètres  
✅ **Pipeline complet** : Du texte à la génération  
✅ **Perplexité** : Métrique de qualité  
✅ **Bonnes pratiques** : Production et limites  

### Prochaines Étapes

1. 🔬 **Expérimenter** avec différents modèles (gpt2, gpt2-medium, etc.)
2. 🎯 **Tester** vos propres use cases
3. 📚 **Approfondir** avec RAG et fine-tuning
4. 🚀 **Déployer** en production avec les bonnes pratiques

---

💡 **N'oubliez pas** : Les LLMs sont des outils puissants mais imparfaits. Comprendre leur fonctionnement vous permet de les utiliser efficacement et responsablement.

🙏 **Merci d'avoir suivi ce notebook !**